In [1]:
import argparse
import time
import torch
from torch.autograd import Variable
import numpy as np
torch.manual_seed(5)
import sys
sys.path.append("..")
# ############################################################################
# param
##############################################################################
batch_size=50
use_cuda=1
##############################################################################
# Load data
##############################################################################
from data_loader import DataLoader
#question
path='/media/kwane/3160053c-937e-4de9-a540-b28bd2802040/kwane/NLP/lstm_text_class/data/rt/'
pretrained_wordvec=np.load(path+'pretrained_wordvec.npy')
data = torch.load(path+'corpus.pt')
max_len = data["max_len"]
vocab_size = data['dict']['vocab_size']
label_size = data['dict']['label_size']

training_data = DataLoader(
             data['train']['src'],
             data['train']['label'],
             data['train']['train_mask'],
             data['train']['train_mask_all'],
             max_len,
             batch_size=batch_size)

validation_data = DataLoader(
              data['valid']['src'],
              data['valid']['label'],
              data['valid']['valid_mask'],
              data['valid']['valid_mask_all'],
              max_len,
              batch_size=batch_size,
              shuffle=False)

In [2]:
# ##############################################################################
# Training
# ##############################################################################
import time
from tqdm import tqdm

train_loss = []
valid_loss = []
accuracy = []

def repackage_hidden(h):
    if type(h) == Variable:
        if use_cuda:
            return Variable(h.data).cuda()
        return Variable(h.data)
    else:
        return tuple(repackage_hidden(v) for v in h)

def evaluate():
    rnn.eval()
    corrects = eval_loss = 0
    _size = validation_data.sents_size
    hidden = rnn.init_hidden()
    for data, label in tqdm(validation_data, mininterval=0.2,
                desc='Evaluate Processing', leave=False):
        hidden = repackage_hidden(hidden)
        pred, hidden = rnn(data, hidden)
        loss = criterion(pred, label)

        eval_loss += loss.data
        corrects += (torch.max(pred, 1)[1].view(label.size()).data == label.data).sum()

    return 1.0*eval_loss[0]/_size, corrects, 1.0*corrects/_size * 100.0, _size

def train():
    rnn.train()
    total_loss = 0
    hidden = rnn.init_hidden()
    for data, label in tqdm(training_data, mininterval=1,
                desc='Train Processing', leave=False):
        optimizer.zero_grad()
        hidden = repackage_hidden(hidden)
        target, hidden = rnn(data, hidden)
        loss = criterion(target, label)

        loss.backward()
        optimizer.step()

        total_loss += loss.data
    return 1.0*total_loss[0]/training_data.sents_size



In [3]:
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.nn import init
import torch.nn.functional as F

import const
class LayerNorm(nn.Module):
    def __init__(self, hidden_size, eps=1e-6):
        super(LayerNorm,self).__init__()
        self.eps = eps
        self.weight = nn.Parameter(torch.ones(hidden_size))
        self.bias = nn.Parameter(torch.zeros(hidden_size))

    def forward(self, input):
        mu = torch.mean(input, dim=-1, keepdim=True)
        sigma = torch.std(input, dim=-1, keepdim=True).clamp(min=self.eps)
        output = (input - mu) / sigma
        return output * self.weight.expand_as(output) + self.bias.expand_as(output)

class LSTM_Text(nn.Module):

    def __init__(self,vocab_size,batch_size,embed_dim,label_size):
        super(LSTM_Text,self).__init__()
        self.vocab_size=vocab_size
        self.embed_dim=embed_dim
        self.hidden_size=128
        self.lstm_layers=1
        self.dropout=0.5
        self.batch_size=batch_size
        self.bidirectional=1
        self.label_size=label_size
        #self.num_directions = 2 if self.bidirectional else 1
        self.num_directions = 2 if 1 else 1

        self.lookup_table = nn.Embedding(self.vocab_size, self.embed_dim,
                                padding_idx=const.PAD)
        self.lstm = nn.LSTM(self.embed_dim,
                            self.hidden_size,
                            self.lstm_layers,
                            dropout=self.dropout,
                            bidirectional=self.bidirectional)
        self.ln = LayerNorm(self.hidden_size*self.num_directions)
        self.logistic = nn.Linear(self.hidden_size*self.num_directions,
                                self.label_size)

        self._init_weights()

    def _init_weights(self, scope=1.):
        self.lookup_table.weight.data.copy_(torch.from_numpy(pretrained_wordvec))
        self.logistic.weight.data.uniform_(-scope, scope)
        self.logistic.bias.data.fill_(0)

    def init_hidden(self):
        num_layers = self.lstm_layers*self.num_directions

        weight = next(self.parameters()).data
        return (Variable(weight.new(num_layers, self.batch_size, self.hidden_size).zero_()),Variable(weight.new(num_layers, self.batch_size, self.hidden_size).zero_()))

    def forward(self, input, hidden):
        #print self.lookup_table.weight.data
        encode = self.lookup_table(input)
        lstm_out, hidden = self.lstm(encode.transpose(0, 1), hidden)
        output = self.ln(lstm_out)[-1]
        return F.log_softmax(self.logistic(output)), hidden

# ##############################################################################
# Build model
# ##############################################################################
import model


rnn = LSTM_Text(vocab_size=vocab_size,batch_size=batch_size,embed_dim=300,label_size=label_size)
if use_cuda:
    rnn = rnn.cuda()

optimizer = torch.optim.Adam(rnn.parameters(), lr=0.001, weight_decay=0.001)
criterion = torch.nn.CrossEntropyLoss()
print rnn


# ##############################################################################
# Save Model
# ##############################################################################
best_acc = None
total_start_time = time.time()

try:
    print('-' * 90)
    for epoch in range(1, 100):
        epoch_start_time = time.time()
        loss = train()
        train_loss.append(loss*1000.)

        print('| start of epoch {:3d} | time: {:2.2f}s | loss {:5.6f}'.format(epoch, time.time() - epoch_start_time, loss))

        loss, corrects, acc, size = evaluate()
        valid_loss.append(loss*1000.)
        accuracy.append(acc)
        print acc
        epoch_start_time = time.time()
        print('-' * 90)
        print('| end of epoch {:3d} | time: {:2.2f}s | loss {:.4f} | accuracy {:.4f}%({}/{})'.format(epoch, time.time() - epoch_start_time, loss, acc, corrects, size))
        print('-' * 90)
        if not best_acc or best_acc < corrects:
            best_acc = corrects
            model_state_dict = rnn.state_dict()
            '''
            model_source = {
                "settings": args,
                "model": model_state_dict,
                "src_dict": data['dict']['train']
            }
            torch.save(model_source, args.save)
            '''
except KeyboardInterrupt:
    print("-"*90)
    print("Exiting from training early | cost time: {:5.2f}min".format((time.time() - total_start_time)/60.0))


Train Processing: 0it [00:00, ?it/s]

LSTM_Text(
  (lookup_table): Embedding(9278, 300, padding_idx=0)
  (lstm): LSTM(300, 128, dropout=0.5, bidirectional=1)
  (ln): LayerNorm(
  )
  (logistic): Linear(in_features=256, out_features=6, bias=True)
)
------------------------------------------------------------------------------------------


/media/kwane/3160053c-937e-4de9-a540-b28bd2802040/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:65: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
Train Processing: 0it [00:00, ?it/s]    

| start of epoch   1 | time: 1.92s | loss 0.095769
30.6
------------------------------------------------------------------------------------------
| end of epoch   1 | time: 0.00s | loss 0.0621 | accuracy 30.6000%(153/500)
------------------------------------------------------------------------------------------


Train Processing: 0it [00:00, ?it/s]   

...

Train Processing: 0it [00:00, ?it/s]   

| start of epoch  97 | time: 0.77s | loss 0.000166
80.4
------------------------------------------------------------------------------------------
| end of epoch  97 | time: 0.00s | loss 0.0185 | accuracy 80.4000%(402/500)
------------------------------------------------------------------------------------------


Train Processing: 0it [00:00, ?it/s]   

| start of epoch  98 | time: 0.77s | loss 0.000079
81.0
------------------------------------------------------------------------------------------
| end of epoch  98 | time: 0.00s | loss 0.0211 | accuracy 81.0000%(405/500)
------------------------------------------------------------------------------------------


| start of epoch  99 | time: 0.77s | loss 0.000097
81.8
------------------------------------------------------------------------------------------
| end of epoch  99 | time: 0.00s | loss 0.0197 | accuracy 81.8000%(409/500)
------------------------------------------------------------------------------------------
